In [17]:
import pandas as pd

def convert_excel_to_html(excel_path, output_html="schedule_tokyo_trip.html"):
    df = pd.read_excel(excel_path)

    # ✅ 지역별 색상 팔레트
    region_colors = {
        "긴자": ("#FFEBEE", "#B71C1C"),      # 빨강
        "도쿄역": ("#E3F2FD", "#1565C0"),    # 파랑
        "마루노우치": ("#FFF8E1", "#EF6C00"),  # 노랑
        "아자부다이": ("#E8F5E9", "#2E7D32"),  # 초록
        "시부야": ("#F3E5F5", "#6A1B9A"),    # 보라
        "아키하바라": ("#FFF3E0", "#E65100"), # 오렌지
        "교바시": ("#E0ECFF", "#1E3A8A"),     # 네이비 블루
    }

    days = df["day"].dropna().unique()
    html = """
    <!DOCTYPE html>
    <html lang="ko">
    <head>
    <meta charset="utf-8">
    <title>Tokyo MVP Global Experience Schedule</title>
    <style>
        body {
            font-family: 'Pretendard', 'Apple SD Gothic Neo', Arial, sans-serif;
            margin: 0;
            background: #f7f7f7;
            color: #222;
            line-height: 1.65;
        }
        header {
            text-align: center;
            padding: 26px 0 14px;
            background: linear-gradient(90deg, #424242, #757575);
            color: #fff;
        }
        h1 {
            margin: 0;
            font-size: 1.8em;
            letter-spacing: 0.5px;
        }
        .tab-bar {
            text-align: center;
            background: #e0e0e0;
            padding: 10px 0;
            position: sticky;
            top: 0;
            z-index: 10;
            box-shadow: 0 2px 4px rgba(0,0,0,0.08);
        }
        .tab-button {
            background: #ffffff;
            border: 1px solid #bdbdbd;
            border-radius: 20px;
            margin: 4px 6px;
            padding: 8px 18px;
            font-size: 0.95em;
            cursor: pointer;
            transition: all 0.2s ease;
            color: #424242;
        }
        .tab-button:hover {
            background: #eeeeee;
        }
        .tab-button.active {
            background: #616161;
            color: #fff;
            border-color: #424242;
            font-weight: 600;
        }
        .day-section {
            display: none;
            max-width: 1080px;
            margin: 0 auto;
            padding: 25px;
        }
        .day-section.active {
            display: block;
        }
        .day-header {
            font-size: 1.4em;
            color: #424242;
            border-left: 6px solid #616161;
            background: #eeeeee;
            padding: 8px 14px;
            border-radius: 6px;
            margin: 20px 0 25px;
            font-weight: 600;
        }

        /* 카드 */
        .schedule-item {
            background: #fff;
            border-radius: 12px;
            padding: 22px 28px 26px 28px;
            margin: 22px 0;
            box-shadow: 0 3px 6px rgba(0,0,0,0.08);
            position: relative;
            transition: all 0.2s ease;
        }
        .schedule-item:hover {
            transform: translateY(-3px);
            box-shadow: 0 6px 12px rgba(0,0,0,0.12);
        }

        /* 지역(region) 라벨 */
        .region-label {
            position: absolute;
            top: 12px;
            right: 26px;
            font-weight: 700;
            font-size: 0.85em;
            padding: 4px 10px;
            border-radius: 6px;
        }

        .location-line {
            font-weight: 600;
            font-size: 1.05em;
            color: #1a1a1a;
            margin-bottom: 10px;
        }
        .location a {
            color: #424242;
            text-decoration: none;
        }
        .location a:hover {
            text-decoration: underline;
        }

        /* 항목별 스티커 박스 */
        .content-box {
            background: #f5f5f5;
            border-left: 5px solid #9e9e9e;
            border-radius: 6px;
            margin: 10px 0;
            padding: 10px 14px;
        }
        .content-title {
            display: inline-block;
            background: #9e9e9e;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .content-text {
            margin: 4px 0 0 4px;
            font-size: 0.95em;
        }

        /* Team Quest */
        .team-quest-box {
            background: #eeeeee;
            border-left: 5px solid #616161;
            border-radius: 6px;
            margin: 14px 0;
            padding: 12px 16px;
        }
        .team-quest-title {
            display: inline-block;
            background: #616161;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .team-quest-title .num {
            margin-left: 2px;
            color: #fff;
            font-weight: 600;
        }

        details {
            margin-top: 14px;
            border-left: 4px solid #bdbdbd;
            background: #f0f0f0;
            padding: 10px 14px;
            border-radius: 6px;
        }
        summary {
            font-weight: 600;
            color: #424242;
            cursor: pointer;
        }
        summary:hover {
            text-decoration: underline;
        }
    </style>

    <script>
        function showDay(dayId) {
            const sections = document.querySelectorAll('.day-section');
            const buttons = document.querySelectorAll('.tab-button');
            sections.forEach(sec => sec.classList.remove('active'));
            buttons.forEach(btn => btn.classList.remove('active'));
            document.getElementById(dayId).classList.add('active');
            document.getElementById('btn-' + dayId).classList.add('active');
        }
        window.onload = () => {
            const first = document.querySelector('.day-section');
            if (first) showDay(first.id);
        };
    </script>
    </head>
    <body>
    <header>
        <h1>🗓️ MVP Global Experience – Tokyo Schedule</h1>
    </header>
    <div class="tab-bar">
    """

    # Day 탭 버튼
    for day in days:
        html += f"<button id='btn-{day}' class='tab-button' onclick=\"showDay('{day}')\">{day}</button>"

    html += "</div>"

    # Day별 섹션 구성
    for day in days:
        html += f"<div class='day-section' id='{day}'>"
        html += f"<div class='day-header'>{day}</div>"
        sub = df[df["day"] == day]

        for _, row in sub.iterrows():
            region = row.get('region')
            desc = f"{row['description']}" if pd.notna(row.get('description')) else ""
            quest_num = int(row['quest_num']) if pd.notna(row.get('quest_num')) else None

            # 지역 라벨 색상 적용
            region_html = ""
            if pd.notna(region) and region in region_colors:
                bg, fg = region_colors[region]
                region_html = f"<div class='region-label' style='background:{bg};color:{fg};'>{region}</div>"

            # 장소명 링크
            location_html = row["location"]
            if pd.notna(row.get("link_map")):
                location_html = f"<a href='{row['link_map']}' target='_blank'>{row['location']}</a>"

            html += f"""
            <div class="schedule-item">
                {region_html}
                <div class="location-line">
                    <span class="location">{location_html}</span> – {desc}
                </div>
            """

            # 일반 항목 박스들
            content_fields = [
                ("고객 경험", "customer_experience"),
                ("공간 및 서비스 특징", "space_features"),
                ("주요 관점", "key_perspective"),
                ("필드 가이드", "field_guide"),
            ]
            for label, field in content_fields:
                value = row.get(field)
                if pd.notna(value):
                    html += f"""
                    <div class='content-box'>
                        <div class='content-title'>{label}</div>
                        <div class='content-text'>{value}</div>
                    </div>
                    """

            # Team Quest 박스 (번호 포함)
            if pd.notna(row.get("team_quest")):
                num_html = f"<span class='num'>#{quest_num}</span>" if quest_num else ""
                html += f"""
                <div class='team-quest-box'>
                    <div class='team-quest-title'>Team Quest {num_html}</div>
                    <div class='content-text'>{row['team_quest']}</div>
                </div>
                """

            if pd.notna(row.get("note")):
                html += f"<details><summary>🗒️ Note</summary><p>{row['note']}</p></details>"

            html += "</div>"

        html += "</div>"

    html += "</body></html>"

    with open(output_html, "w", encoding="utf-8") as f:
        f.write(html)

    return output_html


# ✅ 실행 예시
html_path = convert_excel_to_html("schedule_tokyo_trip.xlsx")
print("✅ HTML 생성 완료:", html_path)

✅ HTML 생성 완료: schedule_tokyo_trip.html
